In [ ]:
import openai
from flask import Flask, request, jsonify

app = Flask(__name__)

# OpenAI API 키 설정
openai.api_key = 'APIKEY'

@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.json
    question = data.get('question', '')

    try:
        # GPT-3를 사용하여 질문에 대한 답변 생성
        response = openai.Completion.create(
          engine="text-davinci-003", 
          prompt=question,
          temperature=0.7,
          max_tokens=150,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
        answer = response.choices[0].text.strip()
    except Exception as e:
        answer = f"죄송합니다, 오류가 발생했습니다: {str(e)}"

    return jsonify({"answer": answer})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


In [ ]:
import openai

openai.api_key = 'APIKEY'

# 파인튜닝 파일 업로드
response = openai.File.create(file=open("finetune_data.jsonl"), purpose='fine-tune')
file_id = response['id']

# 파인튜닝 작업 제출
finetune_response = openai.FineTune.create(
  training_file=file_id,
  model="gpt-3.5-turbo",
  n_epochs=1,
  learning_rate_multiplier=0.1,
  prompt_loss_weight=0.1,
)

print(f"파인튜닝 작업이 시작되었습니다. 작업 ID: {finetune_response['id']}")


In [ ]:
finetune_id = finetune_response['id']
finetune_status = openai.FineTune.retrieve(id=finetune_id)
print(f"파인튜닝 작업 상태: {finetune_status['status']}")


In [ ]:
import openai
from flask import Flask, request, jsonify

app = Flask(__name__)

# OpenAI API 키 설정
openai.api_key = 'APIKEY'

# 파인튜닝된 모델의 ID 설정
finetuned_model_id = "FINETUNEDMODELID"

@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.json
    question = data.get('question', '')

    try:
        # 파인튜닝된 모델을 사용하여 질문에 대한 답변 생성
        response = openai.Completion.create(
          model=finetuned_model_id, # 파인튜닝된 모델의 ID 사용
          prompt=question,
          temperature=0.7,
          max_tokens=150,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
        answer = response.choices[0].text.strip()
    except Exception as e:
        answer = f"죄송합니다, 오류가 발생했습니다: {str(e)}"

    return jsonify({"answer": answer})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets


iris_data = datasets.load_iris()
input_data = iris_data.data
correct = iris_data.target
n_data = len(correct) #샘플 수

#standalization
ave_input = np.average(input_data, axis = 0)
std_input = np.std(input_data, axis = 0)
input_data = (input_data - ave_input) / std_input

#원핫 인코딩 변환
correct_data = np.zeros((n_data, 3))
for i in range(n_data):
    correct_data[i, correct[i]] = 1.0

#훈련 데이터 테스트 데이터 분할
index = np.arange(n_data)
index_train = index[index % 2 == 0]
index_test = index[index % 2 != 0]

input_train = input_data[index_train, :] #훈련 데이터 입력
correct_train = correct_data[index_train, :] #훈련 데이터 정답
input_test = input_data[index_test, :] #테스트 데이터 입력
correct_test = correct_data[index_train, :] #테스트 데이터 정답


n_train = input_train.shape[0] #훈련 데이터 샘플 수
n_test = input_test.shape[0] #테스트 데이터 샘플 수

#입력 은닉 출력층 뉴런 개수
n_in = 4 
n_mid = 25
n_out = 3

wb_width = 0.1
eta = 0.01
epoch = 1600
batch_size = 8
interval = 100


#아다그라드 구현
class BaseLayer:
    def __init__(self, n_upper, n):
        self.w = wb_width * np.random.randn(n_upper, n) #가중치(행렬)
        self.b = wb_width * np.random.randn(n) #편향(벡터)
        
        self.h_w = np.zeros((n_upper, n)) + 1e-8
        self.h_b = np.zeros(n) + 1e-8

    def update(self, eta):
        self.h_w += self.grad_w * self.grad_w
        self.w -= eta / np.sqrt(self.h_w) * self.grad_w
        self.h_b += self.grad_b * self.grad_b
        self.b -= eta / np.sqrt(self.h_b) * self.grad_b


class MiddleLayer(BaseLayer):
    def forward(self, x):
        self.x = x
        self.u = np.dot(x, self.w) + self.b
        self.y = np.where(self.u <= 0, 0, self.u) #ReLU


    def backward(self, grad_y):
        delta = grad_y * np.where(self.u <= 0, 0, 1) #ReLU 미분

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis = 0)

        self.grad_x = np.dot(delta, self.w.T)



class OutputLayer(BaseLayer):
    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b

        #softmax function
        self.y = np.exp(u) / np.sum(np.exp(u), axis = 1, keepdims=True)


    def backward(self, t):
        delta = self.y - t

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis = 0)

        self.grad_x = np.dot(delta, self.w.T)


#드롭아웃 구현

class Dropout:
    def __init__(self, dropout_ratio) -> None:
        self.dropout_ratio = dropout_ratio

    def forward(self, x, is_train):
        if is_train:
            rand = np.random.rand(*x.shape) #난수 행렬
            self.dropout = np.where(rand > self.dropout_ratio, 1, 0)
            self.y = x * self.dropout
        else:
            self.y = (1 - self.dropout_ratio) * x
    def backward(self, grad_y):
        self.grad_x = grad_y * self.dropout
        
#각층 초기화 2

middle_layer_1 = MiddleLayer(n_in, n_mid)
dropout_1 = Dropout(0.5)
middle_layer_2 = MiddleLayer(n_mid, n_mid)
dropout_2 = Dropout(0.5)
output_layer = OutputLayer(n_mid, n_out)

#순전파
def forward_propagation(x, is_train):
    middle_layer_1.forward(x)
    dropout_1.forward(middle_layer_1.y, is_train)
    middle_layer_2.forward(dropout_1.y)
    dropout_2.forward(middle_layer_2.y, is_train)
    output_layer.forward(dropout_2.y)

#역전파
def backpropagation(t):
    output_layer.backward(t)
    dropout_2.backward(output_layer.grad_x)
    middle_layer_2.backward(dropout_2.grad_x)
    dropout_1.backward(middle_layer_2.grad_x)
    middle_layer_1.backward(dropout_1.grad_x)


#가중치, 편향 수정
def uppdate_wb():
    middle_layer_1.update(eta)
    middle_layer_2.update(eta)
    output_layer.update(eta)





#교차 엔트로피 오차 계산
def get_error(t, batch_size):
    return -np.sum(t * np.log(output_layer.y + 1e-7)) / batch_size






#오차 기록용 코드
train_error_x = []
train_error_y = []
test_error_x = []
test_error_y = []



#미니 배치법 구현

n_batch = n_train // batch_size #1에포크당 배치수
for i in range(epoch):

    #오차 계측
    forward_propagation(input_train, is_train=False)
    error_train = get_error(correct_train, n_train)
    forward_propagation(input_test, is_train=False)
    error_test = get_error(correct_test, n_test)

    #오차 기록
    test_error_x.append(i)
    test_error_y.append(error_test)
    train_error_x.append(i)
    train_error_y.append(error_train)

    #경과 표시
    if i % interval == 0:
        print("Epoch: " + str(i) + "/" + str(epoch),
              "Error_train: " + str(error_train),
              "Error_test " + str(error_test))

    #학습
    index_random = np.arange(n_train)
    np.random.shuffle(index_random) #인덱스 랜덤 섞기
    for j in range(n_batch):
            
            #미니배치 샘플 추출
            mb_index = index_random[j * batch_size : (j + 1) * batch_size]
            x = input_train[mb_index, :]
            t = correct_train[mb_index, :]

            #순전파 역전파
            forward_propagation(x, is_train=True)
            backpropagation(t)

            #가중치, 편향 수정
            uppdate_wb()



In [ ]:
#기록된 오차를 그래프로 표시
plt.plot(train_error_x, train_error_y, label = "Train")
plt.plot(test_error_x, test_error_y, label = "Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")

In [ ]:
curl -X POST http://localhost:5000/chatbot -H "Content-Type: application/json" -d "{\"question\":\"지금 날씨 어때?\"}"
